In [3]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt



# from explaneat.core.backprop import NeatNet
# from explaneat.core import backprop
# from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
# from explaneat.core.experiment import ExperimentReporter
# from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score



from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [6]:

USE_CUDA = torch.cuda.is_available()
# USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")
cuda_device = torch.device("cuda:1")

In [7]:
device

# Iris Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [8]:
my_random_seed = 42
random.seed(my_random_seed)

In [9]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [101]:
adult_columns = [
    "age", 
    "workclass",
    "fnlwgt", 
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "gt50k"]

In [102]:
data = pd.read_csv('./../../../data/uci/processed/data/adult/adult.data',
                   names=adult_columns,
                  index_col=False)

In [103]:
data.head()

In [104]:
data.dtypes    

In [105]:
from sklearn.preprocessing import normalize, OneHotEncoder

In [106]:
data.head()

In [110]:
x_cols = data.columns.values.tolist()
y_cols = 'gt50k'
x_cols.remove(y_cols)

xs_raw = data[x_cols]
ys_raw = data[y_cols]

In [111]:
categorical_feature_mask = xs_raw.dtypes==object
numerical_feature_mask = xs_raw.dtypes=="int64"

categorical_cols = xs_raw.columns[categorical_feature_mask].tolist()
numerical_cols = xs_raw.columns[numerical_feature_mask].tolist()

scaler = StandardScaler()

In [145]:
xs = xs_raw.copy()

# OHE categoricals
onehotencoded = pd.get_dummies(xs_raw[categorical_cols])
xs[onehotencoded.columns] = onehotencoded
xs = xs.drop(categorical_cols, axis=1)

## Linear scaling
numericals = xs_raw[numerical_cols].values #returns a numpy array
scaler = StandardScaler()
numericals = scaler.fit_transform(xs_raw[numerical_cols].values)
xs[numerical_cols] = pd.DataFrame(numericals)


####

## Adjust outcome var
ys = data['gt50k'] == ' >50K'

In [148]:
xs

In [149]:
ys

In [150]:
X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=0.15, random_state=42)

Let's have a look at the data we are working with

In [151]:
xs[:5]

In [152]:
ys[:5]

## The competition

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `CrossEntropyLoss` from `PyTorch`

In [153]:
def eval_genomes(genomes, config):
    loss = nn.CrossEntropyLoss()
    loss = loss.to(device)

    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in X_train:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(y_train)))

## The competition

In [1]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 2500, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);


In [155]:
# Use the forest's predict method on the test data
rf_preds = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(rf_preds - y_test)

In [156]:
print('Mean Absolute Error:', round(np.mean(errors), 2))

In [19]:
rf_preds

In [20]:
errors = abs(rf_preds - y_test)

In [21]:
abs(rf_preds.round(0) - y_test)

In [22]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,rf_preds.round(0)))

In [23]:
print(confusion_matrix(y_train, rf.predict(X_train).round(0)))

In [24]:
roc_auc_score(y_test, rf_preds)

# SVM

In [25]:
from sklearn.svm import SVC
svm_model=SVC()


In [26]:
svm_model.fit(X_train, y_train)
svm_preds=svm_model.predict(X_test)
# print(confusion_matrix(y_test,pred))

In [27]:
svm_preds

In [28]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,svm_preds))

In [29]:
print(confusion_matrix(y_train, svm_model.predict(X_train).round(0)))

In [30]:
roc_auc_score(y_test, svm_preds)

# Regression

In [31]:
from sklearn.linear_model import LinearRegression
regression_model=LinearRegression()

In [32]:
regression_model.fit(X_train, y_train)

In [33]:
regression_preds=regression_model.predict(X_test)

In [34]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,regression_preds.round(0)))

In [35]:
print(confusion_matrix(y_train, regression_model.predict(X_train).round(0)))

In [36]:
roc_auc_score(y_test, regression_preds)

# NNs

In [2]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_width=64):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_width) 
        self.fc2 = nn.Linear(hidden_width, hidden_width)
        self.fc3 = nn.Linear(hidden_width, hidden_width)
        self.fc4 = nn.Linear(hidden_width, hidden_width)
        self.fc5 = nn.Linear(hidden_width, output_size)  
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.relu(out)
        out = self.fc5(out)
        return out
class TabularDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, xs, ys):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.xs = xs
        self.ys = ys

    def __len__(self):
        return len(self.xs)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return (x, y)
    



In [130]:
batch_size = 1000
learning_rate = 0.0005
num_epochs = 30

In [131]:
X_train.shape

In [132]:
train_data = TabularDataset(X_train, y_train)
train_loader = DataLoader(train_data, 
                           batch_size=batch_size, 
                           shuffle=True)

validate_data = TabularDataset(X_test, y_test)
validate_loader = DataLoader(dataset = validate_data,
                             batch_size=batch_size, 
                             shuffle=False)

total_step = len(train_loader)

my_random_seed = 42
random.seed(my_random_seed)
nn_model = NeuralNet(9, 1).to(cuda_device)

criterion = nn.BCEWithLogitsLoss().to(cuda_device)
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)  
for epoch in range(num_epochs):
    for i, (xsnn, ysnn) in enumerate(train_loader):  
        # Move tensors to the configured device
        xsnn = xsnn.float().to(cuda_device)
        ysnn = ysnn.view(-1, 1).float().to(cuda_device)

        # Forward pass
        outputs = nn_model(xsnn)
        train_loss = criterion(outputs, ysnn)
        

        # Backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        if (epoch+1) % 50 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, train_loss.item()))


In [133]:
nn_preds = torch.sigmoid( nn_model.forward(torch.from_numpy(X_test).float().to(cuda_device)).to(device)).detach().numpy()
nn_roc_score = roc_auc_score(y_test, nn_preds)
nn_roc_score

# ExplaNEAT

In [134]:
filePath = './../../data/experiments/bchard/experiment-longepochsttsplit-10-0/fullStatus.xplnt'
with gzip.open(filePath, 'rb') as f: 
    data = pickle.load(f)     
    
p, g, ancestry, ancestors, randomState = data
config = p.config
bestNet = neat.nn.FeedForwardNetwork.create(g, config)

explaneat_preds = [bestNet.activate(x)[0] for x in X_test]

In [142]:
print(g.size())

In [143]:
print(g)

In [160]:
def calculate_max_depth(genome):
    connections = {}
    def my_depth(connections, ix):
        if ix < 0:
            return 0
        depths = []
        for c in connections[ix]:
            depths.append(my_depth(connections, c))
        return max(depths) + 1
                
    for key, conn in  genome.connections.items():
        if not key[1] in connections:
            connections[key[1]] = []
        connections[key[1]].append(key[0])
    all_depths = {}
    for c in connections:
        all_depths[c] = my_depth(connections, c)
    return all_depths
print(g.size())
print(calculate_max_depth(g))

In [162]:
config.genome_config.num_inputs

In [165]:
def check_input_connections(genome, config):
    n_inputs = config.genome_config.num_inputs
    checks = {-n: False for n in range(1, n_inputs+1)}
    for c in genome.connections:
        if c[0] < 0:
            checks[c[0]] = True
    print(checks)
    mySum = 0
    for k, v in checks.items():
        if v: 
            mySum += 1
    return mySum
check_input_connections(g, config)

        

In [135]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,np.array(explaneat_preds).round(0)))

In [136]:
roc_auc_score(y_test, explaneat_preds)

In [137]:
train_preds = {
    'svm': svm_model.predict(X_train),
    'regression': regression_model.predict(X_train),
    'rf': rf.predict(X_train),
    'ExplaNEAT': [bestNet.activate(x)[0] for x in X_train],
    'NN': torch.sigmoid( nn_model.forward(torch.from_numpy(X_train).float().to(cuda_device)).to(device)).detach().numpy()
}

plt.figure()
for model, preds in train_preds.items():
    fpr, tpr, thresholds = metrics.roc_curve(y_train, preds)
    auc = metrics.roc_auc_score(y_train, preds)
    plt.plot(fpr, tpr, label='%s ROC (area=%0.3f)' % (model, auc))
# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

In [138]:
test_preds = {
    'svm': svm_model.predict(X_test),
    'regression': regression_model.predict(X_test),
    'rf': rf.predict(X_test),
    'ExplaNEAT': [bestNet.activate(x)[0] for x in X_test],
    'NN': torch.sigmoid( nn_model.forward(torch.from_numpy(X_test).float().to(cuda_device)).to(device)).detach().numpy()
}

plt.figure()
for model, preds in test_preds.items():
    fpr, tpr, thresholds = metrics.roc_curve(y_test, preds)
    auc = metrics.roc_auc_score(y_test, preds)
    plt.plot(fpr, tpr, label='%s ROC (area=%0.3f)' % (model, auc))
# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

In [139]:
all_preds = {
    'svm': svm_preds,
    'regression': regression_preds,
    'rf': rf_preds,
    'ExplaNEAT': explaneat_preds,
    'NN': torch.sigmoid( nn_model.forward(torch.from_numpy(X_test).float().to(cuda_device)).to(device)).detach().numpy()
}

plt.figure()
for model, preds in all_preds.items():
    fpr, tpr, thresholds = metrics.roc_curve(y_test, preds)
    auc = metrics.roc_auc_score(y_test, preds)
    plt.plot(fpr, tpr, label='%s ROC (area=%0.3f)' % (model, auc))
# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

In [140]:
for model, preds in all_preds.items():
    print(model)
    print(len(preds))
    print(metrics.roc_auc_score(y_test, preds))